In [1]:
import os

import numpy as np
import pandas as pd

In [21]:
turnovers = ['THROWAWAY', 'DROP', 'STALL', 'CALLAHAN_THROWN']
pens = ['D_PENALTY_ON_THEM', 'D_PENALTY_ON_US', 'O_PENALTY_ON_THEM', 'O_PENALTY_ON_US']
quarter_ends = ['START_OF_GAME', 'END_OF_Q1', 'HALFTIME', 'END_OF_Q3', 'GAME_OVER']
line_sets = ['SET_D_LINE', 'SET_O_LINE', 'SET_D_LINE_NO_PULL', 'SET_O_LINE_NO_PULL']
timeouts = ['THEIR_MIDPOINT_TIMEOUT', 'OUR_MIDPOINT_TIMEOUT']
other = ['REF_TIMEOUT_DISCUSSION???', 'INJURY_ON_O', 'INJURY_ON_D']
blocks = ['BLOCK', 'CALLAHAN', 'STALL_CAUSED', 'THROWAWAY_CAUSED']
pulls = ['PULL_OUT_OF_BOUNDS', 'PULL_INBOUNDS']


In [3]:
df = pd.read_csv("FlyersWeek3.csv").rename({"Unnamed: 0" : "event_index"}, axis=1)

In [4]:
df

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point
0,0,2894,2022-05-14,242,245,0,242,1,NaN,START_OF_GAME,NaN,NaN,NaN,False,False
1,1,2894,2022-05-14,242,245,1,242,1,NaN,SET_D_LINE,"Grayson Sanner, David Richardson, Justin Allen...",NaN,NaN,False,True
2,2,2894,2022-05-14,242,245,2,242,1,NaN,PULL_INBOUNDS,Rick Hennighausen,-6.89,91.56,False,True
3,3,2894,2022-05-14,242,245,3,242,1,Q1 11:13,SCORED_ON,NaN,NaN,NaN,False,True
4,4,2894,2022-05-14,242,245,4,242,1,NaN,SET_O_LINE,"Alex Davis, Jacob Fairfax, Matt Gouchoe-Hanas,...",NaN,NaN,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559,443,2893,2022-04-29,242,241,443,241,4,NaN,POSSESSION,Khalif El-Salaam,-20.04,37.20,True,False
2560,444,2893,2022-04-29,242,241,444,241,4,NaN,THROWAWAY,NaN,-0.44,96.91,True,False
2561,445,2893,2022-04-29,242,241,445,241,4,Q4 00:04,REF_TIMEOUT_DISCUSSION???,NaN,NaN,NaN,True,False
2562,446,2893,2022-04-29,242,241,446,241,4,NaN,D_PENALTY_ON_US,NaN,NaN,NaN,True,False


In [24]:
df['event_type'].value_counts()
df['event_type'].unique()
df['event_type'].unique()

array(['START_OF_GAME', 'SET_D_LINE', 'PULL_INBOUNDS', 'SCORED_ON',
       'SET_O_LINE', 'POSSESSION', 'D_PENALTY_ON_THEM', 'GOAL',
       'D_PENALTY_ON_US', 'THROWAWAY_CAUSED', 'THROWAWAY',
       'THEIR_MIDPOINT_TIMEOUT', 'SET_D_LINE_NO_PULL', nan,
       'O_PENALTY_ON_THEM', 'BLOCK', 'END_OF_Q1',
       'REF_TIMEOUT_DISCUSSION???', 'O_PENALTY_ON_US',
       'OUR_MIDPOINT_TIMEOUT', 'SET_O_LINE_NO_PULL', 'HALFTIME',
       'END_OF_Q3', 'DROP', 'GAME_OVER', 'PULL_OUT_OF_BOUNDS', 'STALL',
       'INJURY_ON_O', 'INJURY_ON_D', 'STALL_CAUSED', 'CALLAHAN',
       'CALLAHAN_THROWN'], dtype=object)

In [35]:
df.loc[df['event_type'] == "CALLAHAN"]
df.loc[df['event_type'] == "CALLAHAN_THROWN"]
#df.iloc[110:120]
df.iloc[2360:2372]

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,turn,who_turned,reciever
2360,244,2893,2022-04-29,242,241,244,241,3,Q3 07:40,GOAL,Elijah Jaime,-17.64,107.76,True,False,0,0.0,"Jacob Blackman, Alex Liu, Khalif El-Salaam, Ta..."
2361,245,2893,2022-04-29,242,241,245,241,3,NaN,SET_D_LINE,"Jacob Blackman, Alex Liu, Khalif El-Salaam, Ta...",NaN,NaN,False,True,0,0.0,Khalif El-Salaam
2362,246,2893,2022-04-29,242,241,246,241,3,NaN,PULL_INBOUNDS,Khalif El-Salaam,0.65,94.44,False,True,0,0.0,NaN
2363,247,2893,2022-04-29,242,241,247,241,3,NaN,O_PENALTY_ON_THEM,NaN,NaN,NaN,False,True,0,0.0,NaN
2364,248,2893,2022-04-29,242,241,248,241,3,NaN,THROWAWAY_CAUSED,NaN,NaN,NaN,False,True,1,0.0,Kennith Taylor
2365,249,2893,2022-04-29,242,241,249,241,3,NaN,POSSESSION,Kennith Taylor,5.02,20.24,False,True,0,0.0,NaN
2366,250,2893,2022-04-29,242,241,250,241,3,Q3 06:46,CALLAHAN_THROWN,NaN,-1.97,16.89,False,True,0,0.0,"JP Burns, Christian Olsen, Karl Ekwurtzel, Aus..."
2367,251,2893,2022-04-29,242,241,251,241,3,NaN,SET_O_LINE,"JP Burns, Christian Olsen, Karl Ekwurtzel, Aus...",NaN,NaN,True,False,0,0.0,Eli Motycka
2368,252,2893,2022-04-29,242,241,252,241,3,NaN,POSSESSION,Eli Motycka,7.43,18.56,True,False,0,0.0,Austin Taylor
2369,253,2893,2022-04-29,242,241,253,241,3,NaN,POSSESSION,Austin Taylor,-3.43,18.13,True,False,0,0.0,Eli Motycka


In [16]:
df['reciever'] = df['player'].shift(-1)
df['turn'] = np.where(df['event_type'].isin(turnovers), 1, 0)

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,turn,who_turned,reciever
2554,438,2893,2022-04-29,242,241,438,241,4,NaN,NaN,NaN,NaN,NaN,False,True,0,0.0,NaN
2555,439,2893,2022-04-29,242,241,439,241,4,Q4 00:17,SCORED_ON,NaN,NaN,NaN,False,True,0,0.0,"Austin Taylor, Khalif El-Salaam, Bradley Seunt..."
2556,440,2893,2022-04-29,242,241,440,241,4,NaN,SET_O_LINE,"Austin Taylor, Khalif El-Salaam, Bradley Seunt...",NaN,NaN,True,False,0,0.0,Sam Gabrielson
2557,441,2893,2022-04-29,242,241,441,241,4,NaN,POSSESSION,Sam Gabrielson,8.74,13.83,True,False,0,0.0,Austin Taylor
2558,442,2893,2022-04-29,242,241,442,241,4,NaN,POSSESSION,Austin Taylor,-2.92,26.65,True,False,0,0.0,Khalif El-Salaam
2559,443,2893,2022-04-29,242,241,443,241,4,NaN,POSSESSION,Khalif El-Salaam,-20.04,37.20,True,False,0,1.0,NaN
2560,444,2893,2022-04-29,242,241,444,241,4,NaN,THROWAWAY,NaN,-0.44,96.91,True,False,1,0.0,NaN
2561,445,2893,2022-04-29,242,241,445,241,4,Q4 00:04,REF_TIMEOUT_DISCUSSION???,NaN,NaN,NaN,True,False,0,0.0,NaN
2562,446,2893,2022-04-29,242,241,446,241,4,NaN,D_PENALTY_ON_US,NaN,NaN,NaN,True,False,0,0.0,NaN
2563,447,2893,2022-04-29,242,241,447,241,4,NaN,GAME_OVER,NaN,NaN,NaN,True,False,0,NaN,NaN


In [17]:
df['turn'] = np.where(df['event_type'].isin(turnovers), 1, 0)
df['turn'] = np.where(df['event_type'] == "DROP", 0.5, 0)
df['who_turned'] = df['turn'].shift(-1)
df['turn'] = np.where(df['event_type'].isin(turnovers), 1, 0)
df

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,turn,who_turned,reciever
0,0,2894,2022-05-14,242,245,0,242,1,NaN,START_OF_GAME,NaN,NaN,NaN,False,False,0,0.0,"Grayson Sanner, David Richardson, Justin Allen..."
1,1,2894,2022-05-14,242,245,1,242,1,NaN,SET_D_LINE,"Grayson Sanner, David Richardson, Justin Allen...",NaN,NaN,False,True,0,0.0,Rick Hennighausen
2,2,2894,2022-05-14,242,245,2,242,1,NaN,PULL_INBOUNDS,Rick Hennighausen,-6.89,91.56,False,True,0,0.0,NaN
3,3,2894,2022-05-14,242,245,3,242,1,Q1 11:13,SCORED_ON,NaN,NaN,NaN,False,True,0,0.0,"Alex Davis, Jacob Fairfax, Matt Gouchoe-Hanas,..."
4,4,2894,2022-05-14,242,245,4,242,1,NaN,SET_O_LINE,"Alex Davis, Jacob Fairfax, Matt Gouchoe-Hanas,...",NaN,NaN,True,False,0,0.0,Matt Gouchoe-Hanas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559,443,2893,2022-04-29,242,241,443,241,4,NaN,POSSESSION,Khalif El-Salaam,-20.04,37.20,True,False,0,0.0,NaN
2560,444,2893,2022-04-29,242,241,444,241,4,NaN,THROWAWAY,NaN,-0.44,96.91,True,False,1,0.0,NaN
2561,445,2893,2022-04-29,242,241,445,241,4,Q4 00:04,REF_TIMEOUT_DISCUSSION???,NaN,NaN,NaN,True,False,0,0.0,NaN
2562,446,2893,2022-04-29,242,241,446,241,4,NaN,D_PENALTY_ON_US,NaN,NaN,NaN,True,False,0,0.0,NaN


In [7]:
df.loc[df['who_turned'] > 0]


,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,turn,who_turned
40,40,2894,2022-05-14,242,245,40,242,1,NaN,D_PENALTY_ON_US,NaN,NaN,NaN,False,True,0,1.0
53,53,2894,2022-05-14,242,245,53,242,1,NaN,POSSESSION,Matthew Mcknight,9.07,49.77,True,False,0,1.0
60,60,2894,2022-05-14,242,245,60,242,1,NaN,POSSESSION,Dylan Hawkins,23.39,50.65,True,False,0,1.0
64,64,2894,2022-05-14,242,245,64,242,1,NaN,NaN,NaN,NaN,NaN,True,False,0,1.0
74,74,2894,2022-05-14,242,245,74,242,1,NaN,POSSESSION,Andrew Vande Berg,-9.48,56.86,False,True,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405,289,2893,2022-04-29,242,241,289,241,3,NaN,D_PENALTY_ON_THEM,NaN,NaN,NaN,False,True,0,1.0
2433,317,2893,2022-04-29,242,241,317,241,3,NaN,PULL_INBOUNDS,Dean Ramsey,19.01,78.85,False,True,0,1.0
2512,396,2893,2022-04-29,242,241,396,241,4,NaN,O_PENALTY_ON_US,NaN,NaN,NaN,True,False,0,1.0
2518,402,2893,2022-04-29,242,241,402,241,4,NaN,POSSESSION,Bradley Seuntjens,-1.03,41.50,True,False,0,1.0


In [8]:
df.iloc[2510:2515]

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,turn,who_turned
2510,394,2893,2022-04-29,242,241,394,241,4,NaN,POSSESSION,Matt Smith,-6.20,41.50,True,False,0,0.0
2511,395,2893,2022-04-29,242,241,395,241,4,NaN,POSSESSION,Austin Taylor,-0.22,38.44,True,False,0,0.0
2512,396,2893,2022-04-29,242,241,396,241,4,NaN,O_PENALTY_ON_US,NaN,NaN,NaN,True,False,0,1.0
2513,397,2893,2022-04-29,242,241,397,241,4,NaN,THROWAWAY,NaN,-0.22,28.44,True,False,1,0.0
2514,398,2893,2022-04-29,242,241,398,241,4,Q4 03:13,SCORED_ON,NaN,NaN,NaN,True,False,0,0.0


In [9]:
df.iloc[58:68]

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,turn,who_turned
58,58,2894,2022-05-14,242,245,58,242,1,NaN,POSSESSION,Noah Saul,1.50,31.02,True,False,0,0.0
59,59,2894,2022-05-14,242,245,59,242,1,NaN,POSSESSION,Elijah Long,12.20,41.93,True,False,0,0.0
60,60,2894,2022-05-14,242,245,60,242,1,NaN,POSSESSION,Dylan Hawkins,23.39,50.65,True,False,0,1.0
61,61,2894,2022-05-14,242,245,61,242,1,NaN,THROWAWAY,NaN,6.61,90.40,True,False,1,0.0
62,62,2894,2022-05-14,242,245,62,242,1,Q1 04:21,THEIR_MIDPOINT_TIMEOUT,NaN,NaN,NaN,True,False,0,0.0
63,63,2894,2022-05-14,242,245,63,242,1,NaN,SET_D_LINE_NO_PULL,"Charlie McCutcheon, Suraj Madiraju, William Co...",NaN,NaN,True,False,0,0.0
64,64,2894,2022-05-14,242,245,64,242,1,NaN,NaN,NaN,NaN,NaN,True,False,0,1.0
65,65,2894,2022-05-14,242,245,65,242,1,NaN,THROWAWAY_CAUSED,NaN,NaN,NaN,True,False,1,0.0
66,66,2894,2022-05-14,242,245,66,242,1,NaN,POSSESSION,Elijah Long,8.32,44.31,True,False,0,0.0
67,67,2894,2022-05-14,242,245,67,242,1,Q1 03:21,GOAL,Justin Allen,15.41,108.06,True,False,0,0.0


In [10]:
df2 = df.loc[~df['event_type'].isin(pens)].copy()
df2['turn'] = np.where(df2['event_type'].isin(turnovers), 1, 0)
df2['who_turned'] = df2['turn'].shift(-1)

In [11]:
df2.loc[(df2['who_turned'] > 0) | (df2['turn'] > 0)]

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,turn,who_turned
39,39,2894,2022-05-14,242,245,39,242,1,NaN,PULL_INBOUNDS,William Coffin,8.66,104.93,False,True,0,1.0
41,41,2894,2022-05-14,242,245,41,242,1,NaN,THROWAWAY_CAUSED,NaN,NaN,NaN,False,True,1,0.0
53,53,2894,2022-05-14,242,245,53,242,1,NaN,POSSESSION,Matthew Mcknight,9.07,49.77,True,False,0,1.0
54,54,2894,2022-05-14,242,245,54,242,1,NaN,THROWAWAY,NaN,-22.72,107.31,True,False,1,0.0
60,60,2894,2022-05-14,242,245,60,242,1,NaN,POSSESSION,Dylan Hawkins,23.39,50.65,True,False,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,397,2893,2022-04-29,242,241,397,241,4,NaN,THROWAWAY,NaN,-0.22,28.44,True,False,1,0.0
2518,402,2893,2022-04-29,242,241,402,241,4,NaN,POSSESSION,Bradley Seuntjens,-1.03,41.50,True,False,0,1.0
2519,403,2893,2022-04-29,242,241,403,241,4,NaN,STALL,NaN,NaN,NaN,True,False,1,0.0
2559,443,2893,2022-04-29,242,241,443,241,4,NaN,POSSESSION,Khalif El-Salaam,-20.04,37.20,True,False,0,1.0


In [12]:
df2.loc[df2['event_type'] == "DROP"]

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,turn,who_turned
402,402,2894,2022-05-14,242,245,402,242,4,NaN,DROP,Alex Davis,14.04,54.81,True,False,1,0.0
494,53,2894,2022-05-14,242,245,53,245,1,NaN,DROP,Jacob Lane,-12.90,53.11,True,False,1,0.0
735,294,2894,2022-05-14,242,245,294,245,3,NaN,DROP,Carl Johnson,-10.24,54.20,True,False,1,0.0
776,335,2894,2022-05-14,242,245,335,245,3,NaN,DROP,Matt McDonald,14.66,39.81,True,False,1,0.0
937,45,2905,2022-05-07,245,242,45,245,1,NaN,DROP,Jacob Lane,-22.92,78.00,False,True,1,0.0
1223,331,2905,2022-05-07,245,242,331,245,3,NaN,DROP,Tannon Hedges,7.70,70.02,False,True,1,0.0
1277,385,2905,2022-05-07,245,242,385,245,4,NaN,DROP,Robert Alongi,-8.73,106.15,False,True,1,0.0
1430,77,2905,2022-05-07,245,242,77,242,2,NaN,DROP,Jacob Fairfax,24.41,107.52,True,False,1,1.0
1982,270,2893,2022-04-29,242,241,270,242,3,NaN,DROP,Trevor Lynch,2.45,57.47,True,False,1,0.0
